In [0]:
# create sparkSession

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder \
    .appName("Log_Analysis") \
    .config("spark.some.config.option", "value") \
    .getOrCreate()

In [0]:
# write code to get data from azure storage to data bricks

In [0]:
storage_acc_name= "mycon2"
storage_acc_key= "PSzPRZyHTj3FBVVdmKOSCYP9IT8MZzKRF1g3H3nyTuniIrBSJ10NjKgaHetUWLyKcatdWnqr7Wy3+AStOBJ4Yg=="
container= "con2"

In [0]:
spark.conf.set(f"fs.azure.account.key.{storage_acc_name}.blob.core.windows.net",storage_acc_key)

In [0]:
dbutils.fs.ls(f"wasbs://{container}@{storage_acc_name}.blob.core.windows.net/")

[FileInfo(path='wasbs://con2@mycon2.blob.core.windows.net/log_data.csv', name='log_data.csv', size=33574, modificationTime=1709189578000)]

In [0]:
# loading data into databricks

In [0]:
df=spark.read.csv(f"wasbs://{container}@{storage_acc_name}.blob.core.windows.net/",header=True, inferSchema=True)

In [0]:
df.show(10)

+--------------------+---------------+--------------+-----------+----------+
|           Timestamp|     IP Address|    User Agent|HTTP Status|Bytes Sent|
+--------------------+---------------+--------------+-----------+----------+
|2023-12-23 12:33:...| 192.168.252.73|UserAgent-5863|        404|      8989|
|2023-03-14 07:00:...| 192.168.142.94|UserAgent-9655|        200|      4671|
|2024-02-01 01:24:...|192.168.238.242|UserAgent-7686|        200|      2316|
|2023-04-16 19:58:...| 192.168.61.130|UserAgent-6243|        500|      9813|
|2024-02-11 16:13:...|192.168.133.229|UserAgent-6416|        200|      8081|
|2023-07-01 08:43:...|192.168.156.144|UserAgent-3904|        404|       929|
|2023-07-12 13:56:...| 192.168.190.45|UserAgent-2851|        200|      2709|
|2023-06-17 18:06:...| 192.168.42.159|UserAgent-3907|        500|      2934|
|2023-03-02 03:33:...| 192.168.203.85|UserAgent-6360|        200|      7955|
|2023-08-12 04:58:...|192.168.206.168|UserAgent-7090|        200|      9166|

In [0]:
# Instead of explicitly specifying the full path every time to access the data, we can use mounting

In [0]:
storage_acc_name= "mycon2"
storage_acc_key= "PSzPRZyHTj3FBVVdmKOSCYP9IT8MZzKRF1g3H3nyTuniIrBSJ10NjKgaHetUWLyKcatdWnqr7Wy3+AStOBJ4Yg=="
container= "con2"

dbutils.fs.mount(
    source=f"wasbs://{container}@{storage_acc_name}.blob.core.windows.net/",
    mount_point="/mnt/storage",
    extra_configs={f"fs.azure.account.key.{storage_acc_name}.blob.core.windows.net": storage_acc_key}
)

True

In [0]:
dbutils.fs.mounts()

[MountInfo(mountPoint='/mnt/storage', source='wasbs://con2@mycon2.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(mountPoint='/volume', source='DbfsReserved', encryptionType='')]

In [0]:
df = spark.read.csv("/mnt/storage/log_data.csv", header=True, inferSchema=True)

In [0]:
df.show(10)

+--------------------+---------------+--------------+-----------+----------+
|           Timestamp|     IP Address|    User Agent|HTTP Status|Bytes Sent|
+--------------------+---------------+--------------+-----------+----------+
|2023-12-23 12:33:...| 192.168.252.73|UserAgent-5863|        404|      8989|
|2023-03-14 07:00:...| 192.168.142.94|UserAgent-9655|        200|      4671|
|2024-02-01 01:24:...|192.168.238.242|UserAgent-7686|        200|      2316|
|2023-04-16 19:58:...| 192.168.61.130|UserAgent-6243|        500|      9813|
|2024-02-11 16:13:...|192.168.133.229|UserAgent-6416|        200|      8081|
|2023-07-01 08:43:...|192.168.156.144|UserAgent-3904|        404|       929|
|2023-07-12 13:56:...| 192.168.190.45|UserAgent-2851|        200|      2709|
|2023-06-17 18:06:...| 192.168.42.159|UserAgent-3907|        500|      2934|
|2023-03-02 03:33:...| 192.168.203.85|UserAgent-6360|        200|      7955|
|2023-08-12 04:58:...|192.168.206.168|UserAgent-7090|        200|      9166|

In [0]:
df.printSchema()

root
 |-- Timestamp: timestamp (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- User Agent: string (nullable = true)
 |-- HTTP Status: integer (nullable = true)
 |-- Bytes Sent: integer (nullable = true)



In [0]:
display(df)

Timestamp,IP Address,User Agent,HTTP Status,Bytes Sent
2023-12-23T12:33:28.710082Z,192.168.252.73,UserAgent-5863,404,8989
2023-03-14T07:00:28.710082Z,192.168.142.94,UserAgent-9655,200,4671
2024-02-01T01:24:11.710082Z,192.168.238.242,UserAgent-7686,200,2316
2023-04-16T19:58:21.710082Z,192.168.61.130,UserAgent-6243,500,9813
2024-02-11T16:13:27.710082Z,192.168.133.229,UserAgent-6416,200,8081
2023-07-01T08:43:59.710082Z,192.168.156.144,UserAgent-3904,404,929
2023-07-12T13:56:11.710082Z,192.168.190.45,UserAgent-2851,200,2709
2023-06-17T18:06:36.710082Z,192.168.42.159,UserAgent-3907,500,2934
2023-03-02T03:33:53.710082Z,192.168.203.85,UserAgent-6360,200,7955
2023-08-12T04:58:15.710082Z,192.168.206.168,UserAgent-7090,200,9166


In [0]:
# Handling missing values

In [0]:
df = df.na.drop()

In [0]:
# Let's compute basic statistics for numerical columns

In [0]:
df.describe().show()

+-------+-------------+--------------+----------------+-----------------+
|summary|   IP Address|    User Agent|     HTTP Status|       Bytes Sent|
+-------+-------------+--------------+----------------+-----------------+
|  count|          500|           500|             500|              500|
|   mean|         NULL|          NULL|         361.952|         5218.944|
| stddev|         NULL|          NULL|125.575858701333|2886.614195192661|
|    min|192.168.0.131|UserAgent-1010|             200|              114|
|    max|192.168.98.35|UserAgent-9990|             500|             9990|
+-------+-------------+--------------+----------------+-----------------+



In [0]:
# Finding error rates

In [0]:
total_records = df.count()

In [0]:
from pyspark.sql.functions import col
error_records = df.filter((col('HTTP Status') == 404) | (col('HTTP Status') == 500)).count()

In [0]:
error_rate = (error_records / total_records) * 100
error_rate

64.8

In [0]:
# Analyzing trends
# We can analyze trends by grouping data by a certain time interval

In [0]:
trends = df.groupBy(df.Timestamp.cast("date").alias("Date"), "HTTP Status").count().orderBy("Date")

In [0]:
trends.show()

+----------+-----------+-----+
|      Date|HTTP Status|count|
+----------+-----------+-----+
|2023-02-28|        404|    1|
|2023-03-02|        404|    1|
|2023-03-02|        200|    1|
|2023-03-04|        500|    3|
|2023-03-04|        404|    1|
|2023-03-05|        404|    1|
|2023-03-05|        500|    4|
|2023-03-05|        200|    1|
|2023-03-07|        200|    2|
|2023-03-09|        200|    1|
|2023-03-10|        404|    2|
|2023-03-11|        500|    3|
|2023-03-12|        200|    2|
|2023-03-14|        200|    1|
|2023-03-14|        500|    1|
|2023-03-15|        500|    1|
|2023-03-16|        200|    1|
|2023-03-17|        200|    1|
|2023-03-17|        404|    1|
|2023-03-18|        404|    1|
+----------+-----------+-----+
only showing top 20 rows



In [0]:
ip_trends = df.groupBy(df.Timestamp.cast("date").alias("Date"), "IP Address").count().orderBy("Date")

In [0]:
# Trend of Requests from IP Addresses over Time
ip_trends.show()

+----------+---------------+-----+
|      Date|     IP Address|count|
+----------+---------------+-----+
|2023-02-28| 192.168.26.135|    1|
|2023-03-02| 192.168.203.85|    1|
|2023-03-02|192.168.248.143|    1|
|2023-03-04| 192.168.167.93|    1|
|2023-03-04| 192.168.255.42|    1|
|2023-03-04| 192.168.18.201|    1|
|2023-03-04|   192.168.89.0|    1|
|2023-03-05| 192.168.148.60|    1|
|2023-03-05|192.168.125.175|    1|
|2023-03-05| 192.168.43.182|    1|
|2023-03-05| 192.168.115.63|    1|
|2023-03-05| 192.168.31.163|    1|
|2023-03-05| 192.168.176.53|    1|
|2023-03-07| 192.168.234.44|    1|
|2023-03-07|192.168.239.195|    1|
|2023-03-09|192.168.114.147|    1|
|2023-03-10|  192.168.8.109|    1|
|2023-03-10| 192.168.157.39|    1|
|2023-03-11|   192.168.26.5|    1|
|2023-03-11|192.168.177.113|    1|
+----------+---------------+-----+
only showing top 20 rows

